## Configuração do Ambiente (Atualizada)
Assegure-se de instalar versões compatíveis antes de importar qualquer biblioteca:
```bash
%pip install --upgrade --force-reinstall "numpy<2" pandas nltk scikit-learn matplotlib wordcloud pyarrow typing_extensions>=4.8.0 "spacy>=3.6.2"
%python -m spacy download pt_core_news_sm
```

In [ ]:
# Imports e setup
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Download de stopwords
nltk.download('stopwords')

# Carrega modelo spaCy PT
try:
    nlp = spacy.load('pt_core_news_sm')
except OSError:
    from spacy.cli import download as spacy_download
    spacy_download('pt_core_news_sm')
    nlp = spacy.load('pt_core_news_sm')

TypeError: ForwardRef._evaluate() missing 1 required keyword-only argument: 'recursive_guard'

: 

In [ ]:
# Carrega dataset
df = pd.read_csv('../data/comments_dataset_pt_improved.csv')
df.head()

In [ ]:
# Stop-words em Português
stopwords_pt = stopwords.words('portuguese')

In [ ]:
# 1. Análise de Frequência de Termos (Keyword Analysis)
cv = CountVectorizer(stop_words=stopwords_pt)
dtm = cv.fit_transform(df['comentario'])
freq = dtm.sum(axis=0).A1
terms = cv.get_feature_names_out()
top10 = sorted(zip(terms, freq), key=lambda x: x[1], reverse=True)[:10]
import pandas as pd
pd.DataFrame(top10, columns=['Termo','Frequência'])

In [ ]:
# 2. TF-IDF
tfidf_vec = TfidfVectorizer(stop_words=stopwords_pt)
tfidf = tfidf_vec.fit_transform(df['comentario'])
df_tfidf = pd.DataFrame(tfidf.toarray(), columns=tfidf_vec.get_feature_names_out())
df_tfidf.iloc[0].sort_values(ascending=False).head(5)

In [ ]:
# 3. Análise de Sentimento (léxico simples)
positivos = {'ótimo','excelente','adorei','incrível','sensacional','útil','top','fantástico','perfeito'}
negativos = {'ruim','péssimo','problema','lento','difícil','insuportável','chato'}
def pontuar(texto):
    tokens = texto.lower().split()
    return sum(t in positivos for t in tokens) - sum(t in negativos for t in tokens)
df['sentiment_score'] = df['comentario'].apply(pontuar)
df[['comentario','sentiment_score']].head(10)

In [ ]:
# 4. Modelagem de Tópicos (LDA)
lda = LatentDirichletAllocation(n_components=2, random_state=0)
lda.fit(dtm)
topics = [(f'Tópico {i+1}', [terms[j] for j in comp.argsort()[-5:]]) for i, comp in enumerate(lda.components_)]
import pandas as pd
pd.DataFrame(topics, columns=['Tópico','Termos'])

In [ ]:
# 5. Similaridade de Texto
sim = cosine_similarity(tfidf[0], tfidf[1])[0][0]
sim

In [ ]:
# 6. Reconhecimento de Entidades Nomeadas
[[(ent.text, ent.label_) for ent in doc.ents] for doc in nlp.pipe(df['comentario'].head(5))]

In [ ]:
# 7. Codificação Qualitativa de Dados
def categorizar(texto):
    txt = texto.lower()
    if 'tutorial' in txt or 'dica' in txt:
        return 'Dicas'
    if 'ótimo' in txt or 'excelente' in txt:
        return 'Elogio'
    return 'Outro'
df['categoria'] = df['comentario'].apply(categorizar)
df['categoria'].value_counts()

In [ ]:
# 8. Visualização de Texto
import matplotlib.pyplot as plt
from wordcloud import WordCloud

pd.Series(dict(top10)).sort_values().plot.barh(figsize=(6,4), title='Top 10 termos frequentes')
plt.show()

wc = WordCloud(width=400, height=200, stopwords=set(stopwords_pt))
wc.generate(' '.join(df['comentario']))
plt.figure(figsize=(8,4))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()